## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hilo,US,19.7297,-155.0900,78.8,53,20,8.05,few clouds
1,1,Hermanus,ZA,-34.4187,19.2345,59.0,93,100,2.95,overcast clouds
2,2,Yellowknife,CA,62.4560,-114.3525,10.4,48,20,9.22,few clouds
3,3,Padang,ID,-0.9492,100.3543,77.0,94,40,1.14,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,50.0,57,40,5.75,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                283
City                   283
Country                280
Lat                    283
Lng                    283
Max Temp               283
Humidity               283
Cloudiness             283
Wind Speed             283
Current Description    283
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.80,few clouds,19.7297,-155.0900,
3,Padang,ID,77.00,scattered clouds,-0.9492,100.3543,
5,Antofagasta,CL,62.60,broken clouds,-23.6500,-70.4000,
8,Boca Do Acre,BR,70.45,moderate rain,-8.7522,-67.3978,
9,Buchanan,LR,78.80,scattered clouds,5.8808,-10.0467,
12,Port Alfred,ZA,69.44,overcast clouds,-33.5906,26.8910,
13,Arraial Do Cabo,BR,76.26,clear sky,-22.9661,-42.0278,
16,Hualmay,PE,69.03,broken clouds,-11.0964,-77.6139,
17,Rikitea,PF,79.03,clear sky,-23.1203,-134.9692,
28,Namibe,AO,78.84,scattered clouds,-15.1961,12.1522,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hilo,US,78.80,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
3,Padang,ID,77.00,scattered clouds,-0.9492,100.3543,Mercure Padang
5,Antofagasta,CL,62.60,broken clouds,-23.6500,-70.4000,Holiday Inn Express Antofagasta
8,Boca Do Acre,BR,70.45,moderate rain,-8.7522,-67.3978,Hotel Sao Pedro
9,Buchanan,LR,78.80,scattered clouds,5.8808,-10.0467,Kamaneahn Hotel Inc
12,Port Alfred,ZA,69.44,overcast clouds,-33.5906,26.8910,The Halyards Hotel
13,Arraial Do Cabo,BR,76.26,clear sky,-22.9661,-42.0278,Pousada Porto Praia
16,Hualmay,PE,69.03,broken clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
17,Rikitea,PF,79.03,clear sky,-23.1203,-134.9692,Pension Maro'i
28,Namibe,AO,78.84,scattered clouds,-15.1961,12.1522,Hotel Chik Chik Namibe


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))